In [413]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math

# Modelo general

In [414]:
model = gp.Model("Generación de horarios de colegio")
model.setParam('OutputFlag', 1)

M = 10000
C_ = ['A'] # Cursos
I_ = [1] # Niveles
tope_modulos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
J_ = {
    'Lunes': [1, 2, 3, 4, 5, 6], 
    'Martes': [1, 2, 3, 4, 5, 6, 7, 8], 
    'Miércoles': [1, 2, 3, 4, 5, 6], 
    'Jueves': [1, 2, 3, 4, 5, 6, 7, 8], 
    'Viernes': [1, 2, 3, 4, 5, 6]
} # Módulos
combinaciones = [
    (1,2),(3,4),(5,6),(7,8)
]
K_ = [
    'Lenguaje', 'Matemáticas', 'Educación_Física', 'Historia', 'Ciencias_Naturales', 'Inglés', 'Artes_Visuales',
    'Religión', 'Tecnología', 'Música', 'Orientación_Pastoral'
]
D_ = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
P_ = [
    'Alexis_Sanchez', 'Arturo_Vidal', 'Claudio_Bravo', 
    'Gary_Medel', 
    'Harry_Potter', 'Ron_Weasley', 'Hermione_Granger', 'Draco_Malfoy',
    'Harry_Potter2', 'Ron_Weasley2', 'Hermione_Granger2', 'Draco_Malfoy2'
]

clases_por_profesor = {
    'Alexis_Sanchez': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2)],
    'Arturo_Vidal': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1)],
    'Claudio_Bravo': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2)],
    'Gary_Medel': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1)],

    'Harry_Potter': [('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), ('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1)],    
    'Ron_Weasley': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1)],
    'Hermione_Granger': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2)],
    'Draco_Malfoy': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1)],

    'Harry_Potter2': [('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), ('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1)],    
    'Ron_Weasley2': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1)],
    'Hermione_Granger2': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2)],
    'Draco_Malfoy2': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1)],
}

clases_por_nivel = {
    ('Lenguaje', 1): 8,
    ('Matemáticas', 1): 6,
    ('Educación_Física', 1): 4,
    ('Historia', 1): 3,
    ('Ciencias_Naturales', 1): 3,
    ('Inglés', 1): 2,
    ('Artes_Visuales', 1): 2, 
    ('Religión', 1): 2,
    ('Tecnología', 1): 1,
    ('Orientación_Pastoral', 1): 1,
    ('Música', 1): 2, 
    
    ('Lenguaje', 2): 8,
    ('Matemáticas', 2): 6,
    ('Educación_Física', 2): 4,
    ('Historia', 2): 3,
    ('Ciencias_Naturales', 2): 3,
    ('Inglés', 2): 2,
    ('Artes_Visuales', 2): 2, 
    ('Religión', 2): 2,
    ('Tecnología', 2): 1,
    ('Orientación_Pastoral', 2): 1,
    ('Música', 2): 2
}


x = model.addVars(K_, D_, tope_modulos, I_, C_, vtype=GRB.BINARY, name='x')
y = model.addVars(K_, D_, tope_modulos, I_, C_, P_, vtype=GRB.BINARY, name='y')
s = model.addVars(K_, D_, I_, C_, vtype=GRB.BINARY, name='s')

# Restricciones del modelo

##### Una clase por módulo y solo en los que se permite

In [415]:
model.addConstrs((sum(x[k,d,j,i,c] for k in K_) == 1 for d in D_ for j in J_[d] for i in I_ for c in C_),name="RX1");

In [416]:
model.addConstrs((sum(x[k,d,j,i,c] for k in K_) == 0 for d in D_ for j in list(set(tope_modulos) - set(J_[d])) for i in I_ for c in C_),name="RX2");

##### Cumplir con la cantidad de módulos de clases de todos los ramos

In [417]:
model.addConstrs((sum(x[k,d,j,i,c] for d in D_ for j in J_[d]) == clases_por_nivel[k, i] for k in K_ for i in I_ for c in C_),name="RX3");

##### Máximo de dos clases de la misma materia en un día y si es que hay, que sean seguidas

In [418]:
model.addConstrs((sum(x[k,d,j,i,c] for j in J_[d]) <= 2 for k in K_ for d in D_ for i in I_ for c in C_),name="RX4");

In [419]:
model.addConstrs((x[k,d,j,i,c] + x[k,d,z,i,c] <= 1 for k in K_ for d in D_ for i in I_ for c in C_ for j,z in list(set([(a,b) for a in J_[d] for b in J_[d] if a < b]) - set(combinaciones))),name="RX5");

##### Restricciones profesores

Un profesor por clase

In [420]:
model.addConstrs((sum(y[k,d,j,i,c,p] for k in K_ for p in P_) == 1 for d in D_ for j in J_[d] for i in I_ for c in C_),name="RY1");

Solo hay profesor para cierto horario si en ese horario hay clases

In [421]:
model.addConstrs((sum(y[k,d,j,i,c,p] for p in P_) <= x[k,d,j,i,c] for k in K_ for d in D_ for j in J_[d] for i in I_ for c in C_),name="RY2");

Los profesores no hacen los ramos que no dictan

In [422]:
model.addConstrs((y[k,d,j,i,c,p] == 0 for p in P_ for d in D_ for j in J_[d] for c in C_ for k, i in list(set([(a,b) for a in K_ for b in I_]) - set(clases_por_profesor[p]))),name="RY3");

Los profesores solo hacen una clase en un módulo a la vez

In [423]:
model.addConstrs((sum(y[k,d,j,i,c,p] for k in K_ for i in I_ for c in C_) <= 1 for p in P_ for d in D_ for j in J_[d]),name="RY4");

El mismo profesor le hace todas las clases de un ramo a un mismo curso

In [424]:
model.addConstrs((sum(y[k,d,j,i,c,p] for d in D_ for j in J_[d]) <= 1 for i in I_ for c in C_ for k in K_ for p in P_),name="RY5");

##### Funcionamiento variable s

In [425]:
model.addConstrs((M * s[k,d,i,c] >= sum(x[k,d,j,i,c] for j in J_[d]) for k in K_ for d in D_ for i in I_ for c in C_),name="RS1");

In [426]:
model.addConstrs((sum(s[k,d,i,c] for d in D_) <= math.ceil(clases_por_nivel[k,i] / 2) for k in K_ for d in D_ for i in I_ for c in C_),name="RS2");

##### Correr modelo

In [427]:
obj = sum(
    x[k,d,j,i,c] for k in K_ for d in D_ for j in J_[d] for i in I_ for c in C_
)

model.setObjective(obj, GRB.MINIMIZE)
model.update()
# model.optimize()
model.write('model.lp')

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.optimize()
model.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5332 rows, 7205 columns and 21054 nonzeros
Model fingerprint: 0xf5018ef6
Variable types: 0 continuous, 7205 integer (7205 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 5122 rows and 6874 columns
Presolve time: 0.03s
Presolved: 210 rows, 331 columns, 1445 nonzeros
Variable types: 0 continuous, 331 integer (331 binary)

Root relaxation: objective 3.400000e+01, 168 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      34.0000000   34.00000  0.00%     -    0s

Explored 1 nodes (168 simplex iterations) in 0.05 seconds (0.03 work uni

In [428]:
lista_final = []
with open('out.sol', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line[-2] == '1':
            line = line.strip('\n')
            lista_final.append(line)
# print(lista_final)
diccionario_dias = {
    'Lunes': 1,
    'Martes': 2,
    'Miércoles': 3,
    'Jueves': 4,
    'Viernes': 5
}

vuelta = {
    1: 'Lunes',
    2: 'Martes',
    3: 'Miércoles',
    4: 'Jueves',
    5: 'Viernes'
}

nueva_lista = []
profesores = []
for elem in lista_final:
    if elem[0] == "x":
        elem = elem[2:-3]
        elem = elem.split(',')
        nueva_lista.append(elem)
    elif elem[0] == "y":
        elem = elem[2:-3]
        elem = elem.split(',')
        profesores.append(elem)
for i in range(len(nueva_lista)):
    nueva_lista[i][1] = diccionario_dias[nueva_lista[i][1]]
nueva_lista = sorted(nueva_lista, key= lambda x: x[2])
nueva_lista = sorted(nueva_lista, key= lambda x: x[1])
nueva_lista = sorted(nueva_lista, key= lambda x: x[4])

for i in range(len(nueva_lista)):
    nueva_lista[i][1] = vuelta[nueva_lista[i][1]]

for i in range(len(profesores)):
    profesores[i][1] = diccionario_dias[profesores[i][1]]

profesores = sorted(profesores, key= lambda x: x[2])
profesores = sorted(profesores, key= lambda x: x[1])
profesores = sorted(profesores, key= lambda x: x[4])

for i in range(len(profesores)):
    profesores[i][1] = vuelta[profesores[i][1]]




# print(nueva_lista)
# print(profesores)
# for elem in nueva_lista:
#     if elem[3] == '2' and elem[4] == 'A':
#         print(elem)
# for elem in profesores:
#     if elem[0] == 'Lenguaje':
#         print(elem)

In [429]:
from prettytable import PrettyTable

def generar_horario(curso):
    nivel = curso[:-1]
    paralelo = curso[-1]

    clases_curso = []
    dias = []
    ultimo_dia = None
    horario = PrettyTable(dias)
    lista_auxiliar = []
    lista_dia = None
    for elem in profesores:    
        if elem[3] == nivel and elem[4] == paralelo:
            clases_curso.append([elem[0], elem[1], elem[2], elem[5]])
            if elem[1] not in dias:
                dias.append(elem[1])
            if elem[1] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[1]
                lista_dia.append(ultimo_dia)
                elem[0] = elem[0].replace('_', ' ')
                elem[5] = elem[5].replace('_', ' ')
                lista_dia.append(f'{elem[0]} \n {elem[5]}')
            else:
                elem[0] = elem[0].replace('_', ' ')
                elem[5] = elem[5].replace('_', ' ')
                lista_dia.append(f'{elem[0]} \n {elem[5]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]

    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])

    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '1A'
generar_horario(curso)

Horario del curso 1A: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Historia Claudio Bravo,Ciencias Naturales Hermione Granger2,Lenguaje Harry Potter2,Orientación Pastoral Ron Weasley2,Lenguaje Harry Potter2
2,Tecnología Draco Malfoy,Ciencias Naturales Hermione Granger2,Lenguaje Harry Potter2,Ciencias Naturales Hermione Granger,Lenguaje Alexis Sanchez
3,Matemáticas Hermione Granger2,Música Arturo Vidal,Educación Física Gary Medel,Lenguaje Harry Potter,Artes Visuales Alexis Sanchez
4,Matemáticas Claudio Bravo,Música Arturo Vidal,Educación Física Draco Malfoy2,Lenguaje Harry Potter2,Artes Visuales Harry Potter2
5,Educación Física Gary Medel,Inglés Alexis Sanchez,Historia Hermione Granger2,Religión Arturo Vidal,Matemáticas Hermione Granger2
6,Educación Física Gary Medel,Inglés Harry Potter2,Historia Hermione Granger,Religión Arturo Vidal,Matemáticas Hermione Granger2
7,,Lenguaje Harry Potter2,,Matemáticas Claudio Bravo,
8,,Lenguaje Harry Potter2,,Matemáticas Hermione Granger,
